<a href="https://colab.research.google.com/github/achett/IB-Generation/blob/main/IB_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# !pip -q install openai==0.27.0 langchain huggingface_hub
# !pip install --upgrade --quiet  openllm
# !pip install gpt4all
# !pip install xmltodict
# !pip install python-dotenv
# !pip install langchain_experimental
# !pip install llama-cpp-python

In [1]:
import os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain, LLMSummarizationCheckerChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GPT4All
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.pubmed.tool import PubmedQueryRun
import openai
import textwrap
import dotenv
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat
from os.path import expanduser
from langchain_community.llms import LlamaCpp
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

In [9]:
######################
# AZURE OPENAI MODEL
######################
dotenv.load_dotenv('/content/drive/MyDrive/Colab Notebooks/IB Generation/.env')

AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "chatgpt" #GPT turbo
TEMPERATURE = float(os.environ.get("TEMPERATURE"))
if 'AZURE_OPENAI_KEY' not in os.environ:
    raise RuntimeError("No 'AZURE_OPENAI_KEY' env var set.  Please see README.")
if 'AZURE_OPENAI_VERSION' not in os.environ:
    raise RuntimeError("No API version specified, please set it up as an environment variable")

# configure azure openai for langchain and/or llm
openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = AZURE_OPENAI_VERSION # this may change in the future
                                    # for langchain, set this version in environment variables using OPENAI_API_VERSION


llm = AzureChatOpenAI(deployment_name=AZURE_OPENAI_CHATGPT_DEPLOYMENT, temperature=TEMPERATURE, openai_api_key=AZURE_OPENAI_KEY, api_version=AZURE_OPENAI_VERSION)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [ ]:
# ######################
# # GPT4ALL
# ######################
# local_path = (
#     "/content/drive/MyDrive/Colab Notebooks/IB Generation/models/gpt4all-falcon-newbpe-q4_0.gguf"  # replace with your desired local file path
# )

# # Callbacks support token-wise streaming
# callbacks = [StreamingStdOutCallbackHandler()]

# # Verbose is required to pass to the callback manager
# llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# # If you want to use a custom model add the backend parameter
# # Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

In [ ]:
######################
# LLAMA
######################
model_path = expanduser("/content/drive/MyDrive/Colab Notebooks/IB Generation/models/llama-2-7b-chat.Q4_0.gguf")

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.0,
    streaming=False,
    max_tokens=2000
)
llm = Llama2Chat(llm=llm)

In [ ]:


template_messages = [
    SystemMessage(content="Generate a summary of the following text:"),
    HumanMessagePromptTemplate.from_template("{text}"),
]

prompt_template = ChatPromptTemplate.from_messages(template_messages)


chain = LLMChain(llm=llm, prompt=prompt_template)

print(
    chain.run(
        text=content
    )
)

In [ ]:
tokens = content.split()  # Splits on whitespace by default
number_of_tokens = len(tokens)
print(number_of_tokens)

In [17]:
######################
# DATA
######################
with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source.txt', 'r') as file:
    content = file.read()

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target.txt', 'r') as file:
    target = file.read()

content = ' '.join(content.split())
target = ' '.join(target.split())

wrapped_text = textwrap.fill(content,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100,
and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its
toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the
reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period.
Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was
sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and
100 mg/kg groups, no toxicologically significant changes were noted in any examination during the
dosing period. In the 800 mg/kg group, vomiting was observed in males and females on 1 to 5 days
mainly at Weeks 1 and 2 of dosing. Salivation was observed in males and females immediately after
dosing during the dosing period. Decreased body weight was noted in males and females during the
dosing period. Low eryt

In [11]:
######################
# FACTS
######################
llm_prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract the key facts out of this text. Don't include opinions. Give each fact a number and keep them short sentences"),
        ("user", "{prompt}")
    ])

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

facts = chain.invoke({"prompt":content})

wrapped_facts = textwrap.fill(facts,
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)

print(wrapped_facts)

1. Project 1 was administered orally to cynomolgus monkeys for 4 weeks at doses of 0, 10, 30, 100, and 800 mg/kg.
2. Each dose group consisted of 3 male and 3 female monkeys.
3. An additional 3 males and 3 females were added to the 800 mg/kg group for a 4-week recovery period.
4. The control group received a 0.5 w/v% methylcellulose solution.
5. No animal died or was sacrificed due to moribundity during the study.
6. No toxicologically significant changes were observed in the 10, 30, and 100 mg/kg groups.
7. Vomiting, salivation, and decreased body weight were observed in the 800 mg/kg group.
8. Low erythrocyte count, hematocrit value, and hemoglobin concentration were noted in 1 male and 1 female in the 800 mg/kg group.
9. High triglycerides were observed in the 800 mg/kg group.
10. High glucose levels were noted in 1 male in the 800 mg/kg group.
11. High relative liver weight in males and low adrenal weights in 1 female were observed in the 800 mg/kg group.
12. Tmax occurred between


In [12]:
######################
# Abstractive + Zero Shot Prompt
######################
llm_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate a summary of the following text:"),
        ("user", "{prompt}")
    ])

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

facts = chain.invoke({"prompt":content})

wrapped_facts = textwrap.fill(facts,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

print(wrapped_facts)

Project 1 was tested for toxicity in cynomolgus monkeys over a 4-week period with doses ranging from
0 (control) to 800 mg/kg, followed by a 4-week recovery period for the highest dose group. The study
involved equal numbers of male and female monkeys across all dose groups. No deaths or moribundity-
related sacrifices occurred. At doses up to 100 mg/kg, no significant toxicological changes were
observed. However, at 800 mg/kg, adverse effects such as vomiting, salivation, decreased body
weight, alterations in blood parameters (low erythrocyte count, hematocrit value, and hemoglobin
concentration), high triglycerides, high glucose, and changes in liver and adrenal weights were
noted. Toxicokinetic analysis showed that the maximum concentration (Cmax) and area under the curve
(AUC0-24h) of Project 1 increased with dose, with no significant gender differences or marked
differences after repeated dosing. All changes observed during the dosing period were reversible
during the recovery per

In [14]:
######################
# Abstractive + Few Shot Prompt
######################
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source2.txt', 'r', errors='ignore') as file:
    source2 = file.read()
source2 = ' '.join(source2.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/source3.txt', 'r') as file:
    source3 = file.read()
source3 = ' '.join(source3.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target2.txt', 'r') as file:
    target2 = file.read()
target2 = ' '.join(target2.split())

with open('/content/drive/MyDrive/Colab Notebooks/IB Generation/data/target3.txt', 'r') as file:
    target3 = file.read()
target3 = ' '.join(target3.split())

examples = [
    {"input": source2, "output": target2},
    {"input": source3, "output": target3},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}"),
    ])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

llm_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Generate a summary of the following text:"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)


output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

facts = chain.invoke({"input":content})

wrapped_facts = textwrap.fill(facts,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

print(wrapped_facts)

Project 1 was administered orally to cynomolgus monkeys at doses of 0 (vehicle), 10, 30, 100, and
800 mg/kg daily for 4 weeks, with a subsequent 4-week recovery period for the 800 mg/kg group to
assess reversibility of observed toxicities. No deaths or moribundity-related sacrifices occurred.
In the 10, 30, and 100 mg/kg groups, no significant toxicological changes were observed. However, in
the 800 mg/kg group, adverse effects such as vomiting, salivation, decreased body weight,
alterations in blood parameters (low erythrocyte count, hematocrit value, and hemoglobin
concentration), high triglycerides, high glucose, high relative liver weight, and low adrenal
weights were noted. Toxicokinetic analysis showed Tmax between 0.5 and 4 hours, with Cmax and
AUC0-24h increasing dose-dependently, indicating no significant gender differences or marked
differences after repeated dosing. The observed changes during the dosing period were reversible
during the recovery period. The no-observed-adve

In [15]:
######################
# Abstractive + Chain (Facts and Fact Checker)
######################
checker_chain = LLMSummarizationCheckerChain(llm=llm,
                                             verbose=True,
                                             max_checks=2
                                             )

final_summary = checker_chain.run(content)
final_summary

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_summarization_checker/base.py:107: UserWarning: Directly instantiating an LLMSummarizationCheckerChain with an llm is deprecated. Please instantiate with sequential_chain argument or using the from_llm class method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMSummarizationCheckerChain chain...


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given some text, extract a list of facts from the text.

Format your output as a bulleted list.

Text:
"""
Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period. Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and 100 mg/kg groups, no toxicologically significant changes were noted in any examination during the dosing period. In the 800 mg/kg group, vomiting was

'Project 1 was administered orally once daily for 4 weeks at dose levels of 0 (vehicle), 10, 30, 100, and 800 mg/kg to 3 male and 3 female cynomolgus monkeys per group in order to investigate its toxicity. Three males and three females were added to the 800 mg/kg group in order to assess the reversibility of toxicity observed during the dosing period in a subsequent 4-week recovery period. Animals in the control group received 0.5 w/v% methylcellulose solution. No animal died or was sacrificed due to moribundity in any group during the dosing or recovery period. In the 10, 30, and 100 mg/kg groups, no toxicologically significant changes were noted in any examination during the dosing period. In the 800 mg/kg group, vomiting was observed in males and females on 1 to 5 days mainly at Weeks 1 and 2 of dosing. Salivation was observed in males and females immediately after dosing during the dosing period. Decreased body weight was noted in males and females during the dosing period. Low ery

In [16]:
print(len(content))
print(len(final_summary))

2577
2562


In [ ]:
######################
# AGENT: FACT EXTRACTOR + SUMMARIZER
######################
template='''Extract all facts out of this text. Don't include opinions.
Ensure the facts are structured to make sense to a clinical trial investigator.
Then create a condensed summary that is readable and coherent.
You have access to the following tools:

{tools}

Use the following format:

Content: the text you are extracting key information out of
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
Thought: I now know the final answer
Final Answer: the condensed summary

Begin!

Content: {prompt}
Thought:{agent_scratchpad}'''

tools = [PubmedQueryRun()]

llm_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{prompt}")
    ])


agent = create_react_agent(llm, tools, llm_prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_only_outputs=True)

output_parser = StrOutputParser()

chain = llm_prompt | llm | output_parser

agent_summary = agent_executor.invoke({"prompt":content})

agent_summary = textwrap.fill(agent_summary['output'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)

print(agent_summary)

In [ ]:
print(len(content))
print(len(agent_summary))